**version 5.3**

# Generating Poincaré plots

This example shows how to generate Poincaré plots, a.k.a. puncture plots, with ASCOT5. These plots are generated by tracing markers and recording their position each time a marker crosses a poloidal or toroidal plane. They are mainly used to visualize magnetic field structures and particle resonances. Poincare plots can be made for both field lines and particles.

This tutorial consists of three parts:

1. <a href='#walkthrough'>Quick walkthrough</a>
2. <a href='#generating'>Generating the input</a>
3. <a href='#plotting'>Plotting the output</a>

<a id='walkthrough'></a>


## Quick walkthrough

This is a short guide to show how Poincaré plots are generated most of the time.

Begin by generating a sample HDF5 file with the `simpleruns.py` script:  
`python a5py/preprocessing/simpleruns.py`  
No need to run ASCOT5 yet at this point.

Next we will generate marker and options input needed for a Poincaré simulation:  
`python a5py/preprocessing/poincare.py -fn helloworld.h5`  
Now proceed to run ASCOT5:  
`./ascot5_main --in=helloworld.h5 --d=Poincare`  
The `poincare.py` scripts automatically sets the Poincaré input as active, so no need to activate those manually before running ASCOT5.

After the simulation, the Poincaré data is collected in the HDF5 file. To visualize it, open the GUI  
`a5gui helloworld.h5`  
and click the "Orbit" button in the results. You should now see the Poincaré plot.

By default the color in the plot shows field line length (between the initial position and the point) which is usually not very informative and does not separate field lines. By clicking Id/Time field, different field lines are plotted with different colors that are picked randomly from a pool of N~5 colors.

There are different coordinates in which to plot the Poincaré plots that can be chosen from the drop-down menu. The drop-down menu "Poincare" switches between different Poincaré planes (the value corresponds to the "pncrid" field in the orbit output).

<a id='generating'></a>


## Generating the input

For a Poincaré simulation it is necessary to have a dedicated marker and options input. The easiest way to generate these is to run the script found at `a5py/preprocessing/poincare.py`. The arguments are printed if no arguments are given when running the script.

**fn**  
HDF5 file with a magnetic field present where Poincaré inputs are written. Initializing Poincaré markers requires magnetic field and the one that is set as active is used.

**ntrace, rhomin, and rhomax**  
ntrace is the total number of markers to be simulated and these are initially distributed radially evenly between the interval \[rhomin, rhomax\]. Note that if the magnetic field is poorly normalized (there is a "hole" in rho data on the axis when plotted in GUI) this initialization might fail.

**tor0, pol0**  
Initial toroidal and poloidal angle \[deg\] for the markers. If negative, markers are randomly distributed. Usual habit is to initialize markers at the outer mid-plane (pol0 = 0) and randomly in toroidal direction (tor0 = -1).
 
**rhoend**  
Maximum rho value at which marker simulation is terminated. Negative value means markers are traced until they hit the wall.

**time**  
Time slice at which marker simulation begins. Only relevant for time-dependent fields and MHD. Note that for field line marker simulations time is not evolved so the Poincaré plot is the snap shot of the field. For particles this is not the case as the field evolves in time.

**cputmax**  
Maximum CPU time limit for simulating a single marker. Usually it is not necessary to set this value. However, field line tracing uses adaptive time step (so does the particle tracing but this can be changed manually to the fixed step scheme) and so the simulation might get "stuck" if the field has poor quality.

**pitch**  
Marker pitch. Relevant for when particles are simulated. For field lines this only controls if markers are traced along (1) or opposite (-1) to the field lines. *Note that you will need to change this parameter for particle simulations as the pitch equal to unity (default value) is not acceptable.*

**tor, pol**  
Toroidal angles \[deg\] for (R,z)-planes and poloidal angles for (rho,phi)-planes where Poincaré data is collected. Negative value means that particular Poincaré data is not collected. You can set up multiple planes by giving several angles as `-pol "10 20"`. Unless you are doing something fancy, the default values are ok.

**mhd**  
Toggle whether to include MHD modes or not. Note that the field quality might be poor near the core and the separatrix if the boozer data is of poor quality. Adjusting cputmax, rhomin, and rhomax parameters might still in this case enable you to generate a Poincaré plot.

**species**  
Setting this parameter turns the initialized markers from field line tracers to actual particles which are simulated with the adaptive guiding center mode (it is possible to do gyro-orbit simulation but this has to be changed afterwards in the options). This parameter sets mass and charge(state) for the marker, e.g. setting `-species electron` causes markers to be initialized as electrons. See `a5py/physlib/species.py` for a list of particle species.

**energy**  
Sets the particle energy \[eV\]. Required when particles are simulated instead of field lines.

To include Poincaré input generation to your own script, make the following import:

In [ ]:
from a5py.preprocessing.poincare import gen_poincareoptions, gen_poincaremarkers

These functions generate the data but does not write it to HDF5 (unless `desc` argument is given).

<a id='plotting'></a>


## Plotting the output

While GUI is the most obvious choice for the plotting, you might want to include the plotting to your scripts e.g. to make a publication quality plot. The Poincaré data is stored in the orbit output after the simulation.

In [ ]:
from a5py.ascot5io.ascot5 import Ascot

h5    = Ascot("helloworld.h5")
orbit = h5.active.orbit

The exact same plot that is shown in GUI is created by calling the `poincare` function. Note that you can provide `axes` argument if you have an existing figure on which to plot - otherwise the plot is drawn in to a new figure. `equal=True` sets aspect ratio to equal (for Rz plot). What orbits are drawn is controlled with `ids`. When there are many points so that plotting becomes heavy, `prune` can be used to plot only every N:th point.

In [ ]:
orbit.poincare(1, equal=False, axes=None, markersize=1, prune=1,
               ids=np.arange(1,100,5))

The first argument is `pncrid`, i.e., the ID number of the Poincaré plane to be plotted. The indexing starts at zero going through all poloidal planes first and then through the toroidal planes in same order as they were given in the input.

To change the coordinates in which the Poincaré is plotted, two new arguments are needed (the coordinates can be any that is accessable via the orbit output):

In [ ]:
orbit.poincare("R", "phimod", 1, equal=False, axes=None, markersize=1, prune=1,
               ids=np.arange(1,100,5))

Finally, the color can be changed from indicating different field lines to show some quantity e.g. mileage.

In [ ]:
orbit.poincare("R", "phimod", "time", 1, equal=False, axes=None, markersize=1, prune=1,
               ids=np.arange(1,100,5))